# Initial data processing

## Cleaning the data

We are using human-generated neural reconstructions as our ground truth. The first step is to get rid of comments in the header of the file

In [7]:
import os

def get_human_trajectories():
    file_list = glob.glob("../data/human/*.swc")
    return file_list

def remove_comments(fpaths, fnames):
    """SWC files start with comments, remove before proceeding"""
    for i in range(len(fpaths)):

        input = open(fpaths[i], "r")
        outdir = "../data/human_clean/"
        outdir = os.path.abspath(outdir)
        if not os.path.exists(outdir):
            os.mkdir(outdir)
        outfile = os.path.join(outdir, fnames[i])
        output = open(outfile, "w+")

        for line in input:
            if not line.lstrip().startswith("#"):
                output.write(line)

        input.close()
        output.close()

def main():
    # get human trajectories
    fnames = []
    abs_paths = []
    for root, dirs, fnames_ in os.walk("../data/human/"):
        fnames.extend(fnames_)
        for f in fnames_:
            relpath = os.path.join(root, f)
            abs_path = os.path.abspath(relpath)
            abs_paths.append(abs_path)

    remove_comments(abs_paths, fnames)


main()

Next, we need to split each file into its component branches. The SWC files have the following columns:

`node_id type x_coordinate y_coordinate z_coordinate radius parent_node`

First, I convert the SWC file into a linked list.

In [8]:
from collections import defaultdict

def file_to_linked_list(fpath: list):
    
    # https://stackoverflow.com/a/17756005/4212158
    linked_list = defaultdict(list)
    
    input = open(fpath, "r")
    for line in input:
        # note: if the parent node is -1, then the child_node_id is the true identity of the root node
        child_node_id, type_, x_coord, y_coord, z_coord, radius, parent_node = line.split()
        new_node = (child_node_id, x_coord, y_coord, z_coord)
        if len(new_node) != 4:
            raise Exception("faulty node: {}".format(new_node))
        linked_list[parent_node].append(new_node)
    
    input.close()
    return linked_list


## Chopping  reconstructions into individual branches

I start tracing the neuron starting at each root node, which by convention has a parent_node_id of -1. From each root node, I start to grow the branch

In [9]:
def chop_graph(fname: str, linked_list: dict):
    """take a single reconstruction, and chop it up at every fork"""
    assert isinstance(linked_list, dict)
    
    root_nodes = linked_list["-1"]
    for i, root_node in enumerate(root_nodes):
        print("scanning {} root node {} of {}".format(fname, i, len(root_nodes)))
        root_node_id, _x_coord, _y_coord, _z_coord = root_node

        child_nodes = linked_list[root_node_id]
        for j, child_node in enumerate(child_nodes):
            print("starting child {} of {}".format(j, len(child_nodes)))
            child_node_id,  _x_coord, _y_coord, _z_coord = child_node
            branch_name = "_".join([fname, "root{}".format(root_node_id), "child{}".format(child_node_id)])
            grow_branch(linked_list, root_node_id, child_node_id, branch_name)
            
            

I grow each branch until I encounter a fork. Forks are easy to find because the parent node will have more than one child. When the 

In [26]:
def grow_branch(linked_list: dict, root_node_id: str, first_child_node_id: str, branch_name: str):
    """grow the branch until there's a fork"""
    assert isinstance(linked_list, dict)
    
    # initialize the branch
    print("new branch started: {}".format(branch_name))
    branch = [root_node_id, first_child_node_id]
    
    end_found = False
    parent_node_id = first_child_node_id
    while not end_found:
        if (len(branch) % 10000000 == 0):
            print("{} nodes processed".format(len(branch)))
        
        if (len(branch) > 10*len(linked_list)):
            raise Exception("this is growing out of control")
        try:
            child_nodes = linked_list[parent_node_id]
            if len(child_nodes) > 1: # fork found
                print("fork found after {} nodes".format(len(branch)))
                save_branch_as_swc(branch, branch_name)
                for i, child_node in enumerate(child_nodes):
                    child_node_id, _x_coord, _y_coord, _z_coord = child_node
                    child_branch_name = "_".join(branch_name, "grandchild{}".format(child_node_id))
                    grow_branch(linked_list, parent_node_id, child_node)
                end_found = True
            elif len(child_nodes) == 0:  # no more children
                print("end of branch {} found after {} nodes".format(branch_name, len(branch)))
                save_branch_as_swc(branch, branch_name)
                end_found = True
            else:
                child_node = child_nodes[0]
                child_node_id, _x_coord, _y_coord, _z_coord = child_node  # unpack node from list first
                branch.append(child_node)  # should be just 1 node
                parent_node_id = child_node_id
                

SyntaxError: unexpected EOF while parsing (<ipython-input-26-63511dd52195>, line 36)

Finally, save each branch as its own SWC file. Reminder, the column order convention is 

`node_id type x_coordinate y_coordinate z_coordinate radius parent_node`

In [20]:
def save_branch_as_swc(branch, branch_name):   
    outdir = "../data/human_splitted/"
    outdir = os.path.abspath(outdir)
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    outfile = os.path.join(outdir, branch_name, ".swc")
    print("saving SWC {}".format(branch_name))
    output = open(outfile, "w+")
    
    default_radius = 1.0
    default_type = 3
    
    parent_node_id = "-1"
    for i, node in enumerate(branch):
        child_node_id, x, y, z = node
        swc_line = " ".join(child_node_id, default_type, x, y, z, default_radius, parent_node_id)
        output.write(swc_line)
        parent_node_id = child_node_id

    output.close()

Now, we can go through the list of clean files and actually split everything.

In [21]:
clean_files = []
fnames = []
for root, dirs, fnames_ in os.walk("../data/human_clean/"):
        fnames.extend(fnames_)
        for f in fnames_:
            relpath = os.path.join(root, f)
            abs_path = os.path.abspath(relpath)
            clean_files.append(abs_path)
assert len(clean_files) == len(fnames), "# file paths and fnames don't match: {} and {}".format(len(clean_files), len(fnames))

#linked_lists = []
for i in range(len(fnames)):
    print("#{} parsing {} to linked list".format(i, fnames[i]))
    if i < 1: # TODO remove after debugging
        linked_list = file_to_linked_list(clean_files[i])
        print("#{} splitting {} to branches".format(i, fnames[i]))
        chop_graph(fnames[i], linked_list)

#0 parsing 110_ZWX_LijLiu_06072018.ano.swc to linked list
#0 splitting 110_ZWX_LijLiu_06072018.ano.swc to branches
scanning 110_ZWX_LijLiu_06072018.ano.swc root node 0 of 929
starting child 0 of 1
new branch started: 110_ZWX_LijLiu_06072018.ano.swc_root5497882_child5497881
[('5497880', '16633.000', '38326.000', '2461.000')]
[('5497879', '16633.334', '38325.445', '2461.111')]
[('5497878', '16634.666', '38324.668', '2461.889')]
[('5497877', '16635.000', '38324.000', '2462.000')]
[]


IndexError: list index out of range

## concerting SWC files into 3D images

The SWC files can be rendered into volumetric JPEG images by Vaa3D plugins.